<a href="https://colab.research.google.com/github/flexinJ/flexin/blob/main/_pipswithpython_%7C_%F0%9F%94%96_flexin_forex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pips With Python** 💸

**Dependencies**

In [1]:
!pip install hurst yfinance pandas_ta requests pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=4a337ac5fc649617efb38f0ed96f3f992f43a37939508f2b6343a323dade7741
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


**Imports**

In [2]:
import requests
import yfinance as yf
import pandas as pd
import numpy as np
from hurst import compute_Hc
import pandas_ta as ta
from IPython.display import clear_output
from time import sleep

**Data**

In [3]:
def getFinanceDataFrameBy(ticker, period, interval):
  '''
    Get data from Yahoo Finance.
      param ticker: str
      param period: str
      param interval: str
  '''
  try:
    df = yf.Ticker(ticker).history(period, interval)
    df['Ticker'], df['Period'], df['Interval'] = ticker, period, interval
    df.drop(['Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
    df = fixDateTimeColumnIn(df)
    df = log(df)
    df = hurst(df)
    df = smooth(df)
    df = fast(df)
    df = twentyfour(df)
    df = slow(df)
    df = f(df)
    df = adx(df)
    df = cleanNanRowsIn(df)
    return df
  except:
    return f'❌ Error collect data for {ticker}.'

**Fix**

In [4]:
def fixDateTimeColumnIn(dataFrame):
  '''
    Fix datetime.
      param dataframe: pd.DataFrame
  '''
  try:
    if 'Datetime' not in dataFrame.columns:
      dataFrame['Datetime'] = pd.to_datetime(dataFrame.index)
      return dataFrame
    else:
      return dataFrame
  except:
    return f'❌ Error in fix datetime.'

In [5]:
def cleanNanRowsIn(dataFrame):
  '''
    Drop NaN rows.
      param df: pd.DataFrame
  '''
  try:
    dataFrame = dataFrame.dropna()
    return dataFrame
  except:
    return f'❌ Error in drop NaN rows.'

**Tools**

In [6]:
def discord(message):
  '''
    Send a message to a Discord channel using a webhook.
      param message: str
  '''
  url = 'https://discord.com/api/webhooks/1267663408996552816/-tCBhj8C-rbEc7gSKhktIkcmpDs99RjdNJj03MDnnciMNIJABIUTg3GBcRGRxqRC_p2D'
  data = {
      'content': message
      }
  try:
    response = requests.post(url, json=data)
  except:
    return f'❌ Error sending message to discord.'

In [7]:
def transition():
  '''
    Transition.
  '''
  clear_output(wait=True)
  sleep(1)

**Moving verages**

In [8]:
def fast(df):
  '''
    Compute fast exponential moving average.
      param df: pd.DataFrame
  '''
  try:
    df['Fast'] = df['Close'].ewm(50).mean()
    return df
  except:
    return f'❌ Error compute fast exponential moving average.'

In [9]:
def twentyfour(df):
  '''
    Compute shifted fast exponential moving average.
      param df: pd.DataFrame
  '''
  try:
    df['Twentyfour'] = df['Fast'].shift(24)
    return df
  except:
    return f'❌ Error compute shifted fast exponential moving average.'

In [10]:
def slow(df):
  '''
    Compute slow exponential moving average.
      param df: pd.DataFrame
  '''
  try:
    df['Slow'] = df['Close'].ewm(100).mean()
    return df
  except:
    return f'❌ Error compute slow exponential moving average.'

In [11]:
def f(df):
  '''
    Compute flexin exponential moving average.
      param df: pd.DataFrame
  '''
  try:
    df['Flexin'] = df['Close'].ewm(len(df['Close'])/2).mean()
    return df
  except:
    return f'❌ Error compute flexin exponential moving average.'

Quantitative

In [12]:
def log(df):
  '''
    Compute log return.
      param df: pd.DataFrame
  '''
  try:
    df['Log'] = np.log(df['Close']/df['Close'].shift(1))
    return df
  except:
    return f'❌ Error compute log return.'

In [13]:
def hurst(df):
  '''
    Compute hurst exponent.
      param df: pd.DataFrame
  '''
  try:
    H = lambda x: compute_Hc(x)[0]
    df['Hurst'] = df['Log'].rolling(window=100).apply(H)
    return df
  except:
    return f'❌ Error compute hurst exponent.'

In [14]:
def smooth(df, smooth=[5, 25, 50, 75, 100]):
  '''
    Compute smooth hurst exponent of log returns.
      param df: pd.DataFrame
      param smooth: list
  '''
  try:
    for s in smooth:
      df[f'H_{s}'] = df['Hurst'].ewm(s).mean()
      continue
    return df
  except:
    return f'❌ Error compute smooth hurst exponent.'

In [15]:
def adx(df):
  '''
    Compute average directional index.
      param df: pd.DataFrame
  '''
  try:
    adx = ta.adx(df['High'], df['Low'], df['Close'], length=14)
    df = df.join(adx)
    return df
  except:
    return f'❌ Error compute average directional index.'

Info

In [16]:
def long(df):
  '''
    Long.
      param df: pd.DataFrame
  '''
  try:
    type = 'Up-trend'
    ticker = df['Ticker'].iloc[-1]
    period = df['Period'].iloc[-1]
    date = df['Datetime'].iloc[-1]
    interval = df['Interval'].iloc[-1]
    discord(f'📈{type} for next {period}, 🔖{ticker}, 📅{date}, ⌚{interval}, 🌱{hurst.round(2)} | developed by pipswithpython💸.')
    return f'📈{type} for next {period}, 🔖{ticker}, 📅{date}, ⌚{interval}, 🌱{hurst.round(2)} | developed by pipswithpython💸.'
  except:
    return f'❌ Error long position.'

In [17]:
def short(df):
  '''
    Short.
      param df: pd.DataFrame
  '''
  try:
    type = 'Down-trend'
    ticker = df['Ticker'].iloc[-1]
    period = df['Period'].iloc[-1]
    date = df['Datetime'].iloc[-1]
    interval = df['Interval'].iloc[-1]
    hurst = df['Hurst'].iloc[-1]
    discord(f'📉{type} for next {period}, 🔖{ticker}, 📅{date}, ⌚{interval}, 🌱{hurst.round(2)} | developed by pipswithpython💸.')
    return f'📉{type} for next {period}, 🔖{ticker}, 📅{date}, ⌚{interval}, 🌱{hurst.round(2)} | developed by pipswithpython💸.'
  except:
    return f'❌ Error short position.'

In [18]:
def nothinIn(df):
  '''
    No signal.
      param df: pd.DataFrame
  '''
  try:
    ticker = df['Ticker'].iloc[-1]
    period = df['Period'].iloc[-1]
    date = df['Datetime'].iloc[-1]
    interval = df['Interval'].iloc[-1]
    hurst = df['Hurst'].iloc[-1]
    adx = df['ADX_14'].iloc[-1]
    return f'💤Nothin for next {period}, 🔖{ticker}, 📅{date}, ⌚{interval}, 🌱{hurst.round(2)}, ⚡{adx.round(2)} | developed by pipswithpython💸.'
  except:
    return f'❌ Error in no signal function.'

Verify

In [19]:
def hurstExponentConvergenceIn(df):
  '''
    Verify Hurst Exponent Convergence.
      param df: pd.DataFrame
  '''
  v = None
  try:
    if df['Hurst'].iloc[-1]>0.55 and df['H_5'].iloc[-1]>df['H_25'].iloc[-1] and df['H_25'].iloc[-1]>df['H_50'].iloc[-1] and df['H_50'].iloc[-1]>df['H_75'].iloc[-1] and df['H_75'].iloc[-1]>df['H_100'].iloc[-1]:
      v = True
    else:
      v = False
    return v
  except:
    return f'❌ Error in verify function.'

Trend direction

In [20]:
def futureTrendDirectionIn(df):
  '''
    Position.
      param df: pd.DataFrame
  '''
  try:
    if df['Hurst'].iloc[-1]>0.55 and df['Fast'].iloc[-1]>df['Slow'].iloc[-1] and df['ADX_14'].iloc[-1]>25 and df['DMP_14'].iloc[-1]>df['DMN_14'].iloc[-1] and df['Twentyfour'].iloc[-1]>df['Flexin'].iloc[-1]:
      short(df)
    elif df['Hurst'].iloc[-1]>0.55 and df['Fast'].iloc[-1]<df['Slow'].iloc[-1] and df['ADX_14'].iloc[-1]>25 and df['DMP_14'].iloc[-1]<df['DMN_14'].iloc[-1] and df['Twentyfour'].iloc[-1]<df['Flexin'].iloc[-1]:
      long(df)
    elif df['Hurst'].iloc[-1]<0 and df['ADX_14'].iloc[-1]>25 and df['DMP_14'].iloc[-1]>df['DMN_14'].iloc[-1]:
      long(df)
    elif df['Hurst'].iloc[-1]<0 and df['ADX_14'].iloc[-1]>25 and df['DMP_14'].iloc[-1]<df['DMN_14'].iloc[-1]:
      short(df)
    else:
      nothinIn(df)
  except:
    return f'❌ Error in position function.'

In [21]:
def signal(df):
  '''
    Signals.
      param df: pd.DataFrame
  '''
  if hurstExponentConvergenceIn(df) == True:
    print(futureTrendDirectionIn(df))
  else:
    print(nothinIn(df))

App

In [22]:
def run(tickers, period, interval):
  '''
    Flexin system.
      param t: ticker
      param p: period
      param i: interval
  '''
  for ticker in tickers:
    f = getFinanceDataFrameBy(ticker, period, interval)
    signal(f)
    transition()

Variables

In [23]:
wallet = ['BTC-USD',
          'ETH-USD',
          'EURUSD=X',
          'JPY=X',
          'GBPUSD=X',
          'USDCHF=X',
          'CAD=X',
          'CHFJPY=X',
          'EURGBP=X',
          'EURCHF=X',
          'EURJPY=X',
          'EURCAD=X',
          'GBPCHF=X',
          'GBPJPY=X',
          'AUDJPY=X',
          'NZDUSD=X',
          'AUDUSD=X',
          'USDZAR=X']

Loop

In [ ]:
while True:
  run(wallet, '1d', '1m')

💤Nothin for next 1d, 🔖BTC-USD, 📅2024-09-01 16:59:00+00:00, ⌚1m, 🌱0.44, ⚡14.58 | developed by pipswithpython💸.
